# OData Measurement and Test Result Queries with Digital Thread Services

This notebook demonstrates how to use OData queries to retrieve and analyze test measurements, results, and conditions from the NI Measurement Data Store. It shows hierarchical navigation through TestResults → Steps → Measurements.

## Prerequisites

**⚠️ Important:** Run the `publish_sample_data.ipynb` notebook first to create the sample test data that this notebook queries.

- NI Measurement Data Store running and accessible
- Python environment with the `ni.datastore` package
- Sample test data created by running `publish_sample_data.ipynb`

## Setup

Import the required libraries and create both metadata and data store clients.

In [ ]:
from ni.datastore.metadata import MetadataStoreClient
from ni.datastore.data import DataStoreClient
from datetime import datetime

metadata_store_client = MetadataStoreClient()
data_store_client = DataStoreClient()

## Utility Function for Hierarchical Display

This function helps display test results in a tree-like hierarchy showing TestResults → Steps → Measurements.

In [ ]:
def display_test_hierarchy(test_results, max_measurements=3):
    """
    Display test results in a hierarchical tree format:
    TestResult
    ├── Step 1
    │   ├── Measurement 1
    │   └── Measurement 2
    └── Step 2
        └── Measurement 3
    """
    for i, test_result in enumerate(test_results):
        # Display test result info
        status = test_result.outcome.name if hasattr(test_result, 'outcome') and test_result.outcome else "Unknown"
        test_name = test_result.test_result_name or "Unnamed Test"
        print(f"📋 TestResult: {test_name} [{status}]")
        print(f"   ⏱️  Started: {test_result.start_date_time}")
        if hasattr(test_result, 'end_date_time') and test_result.end_date_time:
            # Calculate duration if both start and end times are available
            start_time = test_result.start_date_time
            end_time = test_result.end_date_time
            if start_time and end_time:
                duration = (end_time - start_time).total_seconds()
                print(f"   ⏲️  Duration: {duration:.2f}s")
        
        # Get and display steps
        steps = data_store_client.query_steps(f"$filter=testresultid eq {test_result.test_result_id}")
        steps_list = list(steps)
        
        for j, step in enumerate(steps_list):
            is_last_step = j == len(steps_list) - 1
            step_prefix = "└──" if is_last_step else "├──"
            step_name = step.step_name or f"Step {j+1}"
            # Steps don't have a status field, so we'll skip showing status
            print(f"   {step_prefix} 🔧 Step: {step_name}")
            
            # Get and display measurements for this step
            measurements = data_store_client.query_measurements(f"$filter=stepid eq {step.step_id}")
            measurements_list = list(measurements)
            
            # Limit measurements displayed to avoid overwhelming output
            displayed_measurements = measurements_list[:max_measurements]
            
            for k, measurement in enumerate(displayed_measurements):
                is_last_measurement = k == len(displayed_measurements) - 1 and len(measurements_list) <= max_measurements
                
                if is_last_step:
                    measurement_prefix = "    └──" if is_last_measurement else "    ├──"
                else:
                    measurement_prefix = "│   └──" if is_last_measurement else "│   ├──"
                
                measurement_name = measurement.measurement_name or f"Measurement {k+1}"
                
                # Try to display measurement value if available
                value_info = ""
                if hasattr(measurement, 'value') and measurement.value is not None:
                    unit = getattr(measurement, 'unit', '')
                    value_info = f" = {measurement.value} {unit}".strip()
                
                print(f"   {measurement_prefix} 📊 {measurement_name}{value_info}")
            
            # Show count if there are more measurements
            if len(measurements_list) > max_measurements:
                remaining = len(measurements_list) - max_measurements
                if is_last_step:
                    print(f"    └── ... and {remaining} more measurements")
                else:
                    print(f"│   └── ... and {remaining} more measurements")
        
        # Add spacing between test results
        if i < len(test_results) - 1:
            print()

## Query Test Results

Uses OData queries to retrieve test results with various filtering options.

**Alternative approaches to get test results:**

```python
# Get all test results by finding unique test result IDs from steps
all_steps = data_store_client.query_steps("")
unique_test_result_ids = list(set(step.test_result_id for step in all_steps))
all_test_results = [data_store_client.get_test_result(test_result_id) for test_result_id in unique_test_result_ids]

# Filter test results by name after retrieving them
power_tests = [tr for tr in all_test_results if tr.test_result_name and 'Power' in tr.test_result_name]
amplifier_tests = [tr for tr in all_test_results if tr.test_result_name and 'Amplifier' in tr.test_result_name]

# Filter by operator ID - first get steps, then test results
steps_by_operator = data_store_client.query_steps("$filter=OperatorId eq guid'12345678-1234-1234-1234-123456789abc'")
operator_test_ids = list(set(step.test_result_id for step in steps_by_operator))
operator_tests = [data_store_client.get_test_result(test_id) for test_id in operator_test_ids]

# Get test results from specific steps filtered by step properties
power_steps = data_store_client.query_steps("$filter=contains(Name,'Power')")
power_test_ids = list(set(step.test_result_id for step in power_steps))
power_tests_from_steps = [data_store_client.get_test_result(test_id) for test_id in power_test_ids]
```

In [ ]:
# Get all test results by finding unique test result IDs from steps
all_steps = data_store_client.query_steps("")
unique_test_result_ids = list(set(step.test_result_id for step in all_steps))
all_test_results = [data_store_client.get_test_result(test_result_id) for test_result_id in unique_test_result_ids]

print(f"Found {len(all_test_results)} test results total")
print("\n=== All Test Results Hierarchy ===")
display_test_hierarchy(all_test_results)

print("\n" + "="*50)
print("=== Filtered Results: Power Supply Tests ===")
power_tests = [tr for tr in all_test_results if tr.test_result_name and 'Power' in tr.test_result_name]
if power_tests:
    display_test_hierarchy(power_tests)
else:
    print("No Power Supply tests found")

## Query Steps

Steps are the individual test phases within a test result. Each step can contain multiple measurements.

**Additional step query examples:**

```python
# Filter by step name
initialization_steps = data_store_client.query_steps("$filter=contains(Name,'Initialize')")
measurement_steps = data_store_client.query_steps("$filter=contains(Name,'Measure')")

# Filter by parent test result ID
steps_for_test = data_store_client.query_steps("$filter=testresultid eq {test_result_id}")

# Filter by step type (if available)
# setup_steps = data_store_client.query_steps("$filter=StepType eq 'Setup'")
# measurement_steps = data_store_client.query_steps("$filter=StepType eq 'Measurement'")

# Combine conditions - find measurement steps
measurement_steps = data_store_client.query_steps("$filter=contains(Name,'Measure')")

# Note: Steps do not have a status field in the current data model
```

In [ ]:
# Query all steps
all_steps = list(data_store_client.query_steps(""))

print(f"Found {len(all_steps)} steps total")
print("\nStep summary:")
for step in all_steps[:10]:  # Show first 10 steps
    step_name = step.step_name or "Unnamed Step"
    # Steps don't have a status field, so we'll skip showing status
    print(f"  🔧 {step_name}")

if len(all_steps) > 10:
    print(f"  ... and {len(all_steps) - 10} more steps")

print("\n=== Steps containing 'Voltage' ===")
voltage_steps = list(data_store_client.query_steps("$filter=contains(Name,'Voltage')"))
for step in voltage_steps:
    step_name = step.step_name or "Unnamed Step"
    # Steps don't have a status field, so we'll skip showing status
    print(f"  🔧 {step_name}")

## Query Measurements

Measurements are the individual data points collected during test steps.

**Additional measurement query examples:**

```python
# Filter by measurement name
voltage_measurements = data_store_client.query_measurements("$filter=contains(Name,'Voltage')")
current_measurements = data_store_client.query_measurements("$filter=contains(Name,'Current')")
frequency_measurements = data_store_client.query_measurements("$filter=contains(Name,'Frequency')")

# Filter by parent step ID
step_measurements = data_store_client.query_measurements("$filter=StepId eq guid'12345678-1234-1234-1234-123456789abc'")

# Filter by measurement value (if available)
# high_voltage = data_store_client.query_measurements("$filter=Value gt 10.0")
# low_current = data_store_client.query_measurements("$filter=Value lt 0.1")

# Filter by unit (if available)
# voltage_units = data_store_client.query_measurements("$filter=Unit eq 'V'")
# current_units = data_store_client.query_measurements("$filter=Unit eq 'A'")

# Combine conditions - find high voltage measurements
high_voltages = data_store_client.query_measurements("$filter=contains(Name,'Voltage') and Value gt 5.0")
```

In [ ]:
# Query all measurements
all_measurements = list(data_store_client.query_measurements(""))

print(f"Found {len(all_measurements)} measurements total")
print("\nMeasurement summary:")
for measurement in all_measurements[:15]:  # Show first 15 measurements
    name = measurement.measurement_name or "Unnamed Measurement"
    
if len(all_measurements) > 15:
    print(f"  ... and {len(all_measurements) - 15} more measurements")

print("\n=== Voltage Measurements ===")
voltage_measurements = data_store_client.query_measurements("$filter=contains(Name,'Voltage')")
for measurement in voltage_measurements:
    name = measurement.measurement_name or "Unnamed Measurement"
    print(f"  ⚡ {name}")

print("\n=== Current Measurements ===")
current_measurements = data_store_client.query_measurements("$filter=contains(Name,'Current')")
for measurement in current_measurements:
    name = measurement.measurement_name or "Unnamed Measurement"
    print(f"  🔌 {name}")

## Query Published Conditions

Published conditions represent test conditions or environmental factors during testing.

**Additional condition query examples:**

```python
# Filter by condition name
temperature_conditions = data_store_client.query_conditions("$filter=contains(Name,'Temperature')")
humidity_conditions = data_store_client.query_conditions("$filter=contains(Name,'Humidity')")

# Filter by parent step ID
step_conditions = data_store_client.query_conditions("$filter=StepId eq guid'12345678-1234-1234-1234-123456789abc'")

# Filter by condition value (if available)
# high_temp = data_store_client.query_conditions("$filter=Value gt 25.0")

# Combine conditions
temp_range = data_store_client.query_conditions("$filter=contains(Name,'Temperature') and Value gt 20.0")
```

In [ ]:
# Query all published conditions
all_conditions = list(data_store_client.query_conditions(""))

print(f"Found {len(all_conditions)} published conditions total")

if all_conditions:
    print("\nPublished conditions summary:")
    for condition in all_conditions[:10]:  # Show first 10 conditions
        name = condition.condition_name if hasattr(condition, 'condition_name') else "Unnamed Condition"
        print(f"  🌡️ {name}")
        
    if len(all_conditions) > 10:
        print(f"  ... and {len(all_conditions) - 10} more conditions")
else:
    print("No published conditions found in the sample data")

## Advanced Filtering Examples

Here are some more complex filtering scenarios combining multiple criteria.

In [ ]:
print("=== Advanced Filtering Examples ===")

# Find failed test results by checking outcome
print("\n1. Failed Tests:")
failed_tests = [tr for tr in all_test_results if hasattr(tr, 'outcome') and tr.outcome and hasattr(tr.outcome, 'name') and tr.outcome.name == 'Failed']
for test in failed_tests:
    test_name = test.test_result_name or "Unnamed Test"
    print(f"  ❌ {test_name} - Failed at {test.start_date_time}")

if not failed_tests:
    print("  ✅ No failed tests found!")

# Find passed test results  
print("\n2. Passed Tests:")
passed_tests = [tr for tr in all_test_results if hasattr(tr, 'outcome') and tr.outcome and hasattr(tr.outcome, 'name') and tr.outcome.name == 'Passed']
for test in passed_tests:
    test_name = test.test_result_name or "Unnamed Test"
    duration = ""
    if hasattr(test, 'start_date_time') and hasattr(test, 'end_date_time') and test.start_date_time and test.end_date_time:
        duration_seconds = (test.end_date_time - test.start_date_time).total_seconds()
        duration = f" ({duration_seconds:.2f}s)"
    print(f"  ✅ {test_name}{duration}")

# Find measurements with specific patterns
print("\n3. Power-related Measurements:")
power_measurements = list(data_store_client.query_measurements("$filter=contains(Name,'Power') or contains(Name,'Voltage') or contains(Name,'Current')"))
for measurement in power_measurements[:5]:  # Show first 5
    name = measurement.measurement_name or "Unnamed Measurement"
    print(f"  ⚡ {name}")

if len(power_measurements) > 5:
    print(f"  ... and {len(power_measurements) - 5} more power measurements")

# Show test result summary statistics
print("\n4. Test Summary Statistics:")
total_tests = len(all_test_results)
passed_count = len(passed_tests)
failed_count = len(failed_tests)
running_tests = [tr for tr in all_test_results if hasattr(tr, 'outcome') and tr.outcome and hasattr(tr.outcome, 'name') and tr.outcome.name == 'Running']
running_count = len(running_tests)

print(f"  📊 Total Tests: {total_tests}")
print(f"  ✅ Passed: {passed_count} ({passed_count/total_tests*100:.1f}%)" if total_tests > 0 else "  ✅ Passed: 0")
print(f"  ❌ Failed: {failed_count} ({failed_count/total_tests*100:.1f}%)" if total_tests > 0 else "  ❌ Failed: 0")
print(f"  🔄 Running: {running_count} ({running_count/total_tests*100:.1f}%)" if total_tests > 0 else "  🔄 Running: 0")

## Summary

This notebook demonstrated comprehensive OData querying for test data and measurements in the Digital Thread Services:

### Data Entities Covered:
- **Test Results** - Complete test executions with outcome and timing
- **Steps** - Individual phases within tests
- **Measurements** - Actual data points collected during testing
- **Published Conditions** - Environmental and test conditions

### Key Query Features:
- **Hierarchical Navigation** - TestResults → Steps → Measurements
- **Outcome Filtering** - Finding passed, failed, or running tests (using `outcome` field)
- **Content Filtering** - Searching by name patterns and values
- **Statistical Analysis** - Summary statistics and reporting
- **Visual Display** - Tree-like hierarchical output format

### OData Operators Used:
- `$filter` with lowercase field names (`testresultid`, `stepid`)
- `contains()`, `startswith()`, `endswith()` for string matching
- `eq`, `gt`, `lt` for exact matches and comparisons
- `and`, `or` for combining multiple conditions

### Data Model Notes:
- **TestResult** objects have an `outcome` field (not `status_type`)
- **Step** objects do not have a status field in the current model
- **GUID fields** should be referenced using lowercase names without the `guid` prefix

### Practical Applications:
- **Test Results Analysis** - Finding failed tests for debugging
- **Performance Monitoring** - Tracking test execution times
- **Data Mining** - Extracting specific measurement types
- **Quality Reporting** - Statistical summaries of test outcomes

This completes the trilogy of OData query notebooks for the NI Measurement Data Store!